In [1]:
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords

In [2]:
nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\clair\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
#combining cleaned_tokens into sentences for LDA
import ast
df = pd.read_csv("dataset_cleaned_NER.csv")

text_column = df['cleaned_tokens']

documents = []

for word_list_str in text_column:
    # Convert the string representation of a list into an actual list
    word_list = ast.literal_eval(word_list_str)
    
    # Join the words into a sentence
    sentence = ' '.join(word_list)
    
    # Append the sentence to the documents list
    documents.append(sentence)

for doc in documents:
    print(doc)

starbucks violate federal labor law increase wag offer new perk benefit non union employees national labor relations board judge find thursday decision latest series nlrb rule find starbucks violate labor law efforts stop unions form coffee shop issue heart case whether current board law starbucks entitle explicitly reward employees participate union activity falsely tell workers federal labor law force take action write administrative law judge mara louise anzalone
first suspect plead guilty singapore largest money launder case convict sentence months jail district court tuesday apr su wenqiang admit charge money launder possess proceed illegal remote gamble offences lie get work pass wife billion us billion assets seize freeze relation case likely make one largest money launder operations world su among suspect arrest simultaneous police raid last august cambodian national whose passport state fujian nab good class bungalow along lewis road bukit timah 3 billion 2.2 billion
meta fin 

In [30]:
from sklearn.feature_extraction.text import CountVectorizer

# Initialize the vectorizer
vectorizer = CountVectorizer()

# Transform the documents into a document-term matrix (X)
X = vectorizer.fit_transform(documents)

print(X.shape)  # Check if it's a valid 2D matrix

(1509, 14980)


In [38]:
from sklearn.decomposition import LatentDirichletAllocation

# Initialize the LDA model with a chosen number of topics
lda = LatentDirichletAllocation(n_components=10, random_state=45)# 5 topics, for example

# Fit the LDA model to the document-term matrix
topics = lda.fit_transform(X)

# Print the topic distribution for each document
print(topics)

# Print the top words for each topic
feature_names = vectorizer.get_feature_names_out()
for topic_idx, topic in enumerate(lda.components_):
    print(f"Topic #{topic_idx}:")
    print(" ".join([feature_names[i] for i in topic.argsort()[:-10 - 1:-1]]))

# lda = LatentDirichletAllocation(n_components=8, random_state=42)  # Adjust `n_components` to the number of topics you want
# lda_topics = lda.fit_transform(X)

# # Step 3: Assign each document to the most dominant topic
# # Find the topic with the highest weight for each document
# topic_assignments = lda_topics.argmax(axis=1)

# # Create a new dataframe with topics assigned to each document
# df['Assigned_Topic'] = topic_assignments

# # Preview the assigned topics
# print(df[['cleaned_tokens', 'Assigned_Topic']].head())

[[0.00144978 0.00144971 0.1828591  ... 0.00144963 0.00144956 0.0014498 ]
 [0.00129902 0.00129899 0.00129894 ... 0.00129908 0.00129909 0.34109999]
 [0.00108717 0.00108712 0.00108726 ... 0.00108705 0.00108728 0.00108708]
 ...
 [0.42308307 0.00137025 0.00137012 ... 0.56595546 0.00137018 0.00137018]
 [0.00161342 0.00161333 0.98548085 ... 0.00161312 0.00161325 0.00161311]
 [0.00106403 0.0010641  0.00106409 ... 0.001064   0.0010642  0.00106406]]
Topic #0:
say sanction state also military people two include russia russian
Topic #1:
say year per cent would company make time take last
Topic #2:
say china singapore trade minister mr state also president sign
Topic #3:
say year mr use company one group singapore share come
Topic #4:
say time one first year us world start openai microsoft
Topic #5:
say million bank also company singapore data us service billion
Topic #6:
say singapore year ai university new hospital use team one
Topic #7:
say singapore two time find us year years university millio

In [ ]:
# def clean_text(text):
#     text = str(text).lower()  # Convert to lowercase
#     text = re.sub(r'\W', ' ', text)  # Remove special characters
#     text = re.sub(r'\s+', ' ', text)  # Remove extra spaces
#     text = ' '.join([word for word in text.split() if word not in stop_words])  # Remove stopwords
#     return text


# df['cleaned_text'] = df['Text'].apply(clean_text)  # Assuming text column is named "text"

In [10]:
# from sklearn.feature_extraction.text import CountVectorizer

# vectorizer = CountVectorizer(max_features=5000)  # Limit vocabulary size
# X = vectorizer.fit_transform(df['cleaned_text'])  # Convert text to numerical format

In [12]:
print(df['cleaned_text'].head())

0    starbucks violated federal labor law increased...
1    first suspect plead guilty singapore largest m...
2    meta fined record breaking 1 2 billion 1 3 bil...
3    singapore 45 year old man linked singapore lar...
4    department education imposed record 14 million...
Name: cleaned_text, dtype: object


BERT for Unsupervised Text Classification

In [40]:
from transformers import BertTokenizer, BertModel
import torch
from sklearn.cluster import KMeans
import pandas as pd
import ast

c:\Users\clair\AppData\Local\Programs\Python\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [41]:

# Load the BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Load your dataset
df = pd.read_csv("dataset_cleaned_NER.csv")

# Preprocess the 'cleaned_tokens' column
text_column = df['cleaned_tokens']

# Combine the tokens into sentences
documents = []
for index, row in text_column.items():
    word_list = ast.literal_eval(row)
    sentence = ' '.join(word_list)
    documents.append(sentence)

# Tokenize the text data using BERT tokenizer
inputs = tokenizer(documents, padding=True, truncation=True, return_tensors="pt", max_length=512)

# Get BERT embeddings for each document (the output of the last hidden layer)
with torch.no_grad():
    outputs = model(**inputs)
    embeddings = outputs.last_hidden_state.mean(dim=1)  # Average the token embeddings to get a sentence embedding

# Use KMeans clustering to group the documents
num_clusters = 5  # Adjust the number of clusters based on your dataset
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(embeddings)

# Assign each document to a cluster
df['Cluster'] = kmeans.labels_

# Preview the clustering results
print(df[['cleaned_tokens', 'Cluster']].head())

c:\Users\clair\AppData\Local\Programs\Python\Python312\Lib\site-packages\huggingface_hub\file_download.py:140: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\clair\.cache\huggingface\hub\models--bert-base-uncased. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)
c:\Users\clair\AppData\Local\Programs\Python\Python312\Lib\site-packages\joblib\externals\loky\ba

                                      cleaned_tokens  Cluster
0  ['starbucks', 'violate', 'federal', 'labor', '...        2
1  ['first', 'suspect', 'plead', 'guilty', 'singa...        2
2  ['meta', 'fin', 'record', 'break', 'billion', ...        3
3  ['singapore', 'year', 'old', 'man', 'link', 's...        2
4  ['department', 'education', 'impose', 'record'...        2


BERT with SBERT

In [50]:
import pandas as pd
import ast
import torch
from sentence_transformers import SentenceTransformer
from sklearn.cluster import KMeans
import umap.umap_ as umap

# Load dataset
df = pd.read_csv("dataset_cleaned_NER.csv")

# Convert token lists into sentences
documents = [" ".join(ast.literal_eval(row)) for row in df["cleaned_tokens"]]

# Load Sentence-BERT (better for clustering)
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Get document embeddings
embeddings = sbert_model.encode(documents, convert_to_tensor=True)

# Reduce dimensions using UMAP
umap_model = umap.UMAP(n_components=10, random_state=42)
embeddings_reduced = umap_model.fit_transform(embeddings.cpu().numpy())

# Clustering using KMeans
num_clusters = 5  # Can be adjusted
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
df["Cluster"] = kmeans.fit_predict(embeddings_reduced)

# Save results
df.to_csv("clustered_data.csv", index=False)

print(df[["cleaned_tokens", "Cluster"]].head())

c:\Users\clair\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\clair\AppData\Local\Programs\Python\Python312\Lib\site-packages\umap\umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


                                      cleaned_tokens  Cluster
0  ['starbucks', 'violate', 'federal', 'labor', '...        1
1  ['first', 'suspect', 'plead', 'guilty', 'singa...        4
2  ['meta', 'fin', 'record', 'break', 'billion', ...        1
3  ['singapore', 'year', 'old', 'man', 'link', 's...        4
4  ['department', 'education', 'impose', 'record'...        4


In [51]:
# Save the NER data to a new CSV
output_path = '../data/processed/SBERTModel.csv'
df.to_csv(output_path, index=False)
print(f"\nCleaned data saved to: {output_path}")


Cleaned data saved to: ../SBERTModel.csv


In [1]:
#visualisation
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.decomposition import PCA
from sentence_transformers import SentenceTransformer

# Load the labeled dataset
df_sbert = pd.read_csv("SBERTModel_Labeled.csv")

# Load Sentence-BERT model
sbert_model = SentenceTransformer("all-MiniLM-L6-v2")

# Generate SBERT embeddings from the text column
documents = df_sbert["Text"].astype(str).tolist()
embeddings = sbert_model.encode(documents, convert_to_tensor=True)

# Reduce dimensions using PCA
pca = PCA(n_components=2, random_state=42)
embeddings_2d_pca = pca.fit_transform(embeddings.cpu().numpy())

# Add PCA coordinates to the dataframe
df_sbert["PCA_1"] = embeddings_2d_pca[:, 0]
df_sbert["PCA_2"] = embeddings_2d_pca[:, 1]

# Generate clustering visualization using PCA
plt.figure(figsize=(10, 6))
sns.scatterplot(
    x="PCA_1", y="PCA_2", hue="Cluster_Label", palette="tab10", data=df_sbert, alpha=0.7
)
plt.title("2D Visualization of Clusters (PCA)")
plt.legend(bbox_to_anchor=(1.05, 1), loc="upper left")
plt.xlabel("PCA Dimension 1")
plt.ylabel("PCA Dimension 2")
plt.show()

ModuleNotFoundError: No module named 'sentence_transformers'

BERT + LDA